In [1]:
import re
import os, sys
import pandas as pd
import random
import jieba

In [13]:
csv_data = pd.read_csv("sqlResult_1558435.csv", usecols=["id","title","content"], encoding="gb18030")
articles = csv_data["content"]
titles = csv_data["title"].tolist()
doc_ids = csv_data["id"].tolist()

In [3]:
def read_articles(articles):
    docs_idx_dict = {}
    for idx, content in enumerate(articles):
        if pd.isna(content): continue
        try:
            content = content.replace("\u3000", "").strip()
            content = re.sub("\t", " ", content)
            content = re.sub("\s+", " ", content)
            content = content.replace(r"\n", "\n").replace(r"\\n", "").replace("\r\n", "\n").replace("\n\n", "\n")
            if not content or content == "\n": continue
            docs_idx_dict[idx] = content
        except Exception as e:
            print(str(e) + "\n" + str(content))
    return docs_idx_dict

In [4]:
def ssplit(text):
    text = re.sub('([。！？\?])([^”’])', r"\1\n\2", text)
    text = re.sub('(\.{6})([^”’])', r"\1\n\2", text)
    text = re.sub('(\…{2})([^”’])', r"\1\n\2", text)
    text = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', text)
    return text

In [5]:
docs_dict = read_articles(articles)
csv_list = []
for idx, content in docs_dict.items():
    content_tokenized = []
    content = ssplit(content).split("\n")
    for line in content:
        if not line or line == "\n": continue
        line = " ".join(list(jieba.cut(line, use_paddle=True)))
        content_tokenized.append(line)
    doc_info_dict = {"doc_id": idx, "title": titles[idx], "content": "\n".join(content_tokenized)}
    csv_list.append(doc_info_dict)
result_data = pd.DataFrame(csv_list, columns=["doc_id", "title", "content"])

Building prefix dict from the default dictionary ...
Loading model from cache /home/fxz/anaconda3/envs/text_preprocess/lib/python3.6/site-packages/jieba/cache/jieba.cache
Loading model cost 0.837 seconds.
Prefix dict has been built successfully.


In [6]:
result_data.to_csv("news_tokenized.csv",index=False,sep=',')

In [16]:
result_data.isnull().sum()

doc_id     0
title      0
content    0
dtype: int64